In [1]:
from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
import time

import numpy as np
import os
import sys

# QuOCS and Qudi

### Creation of the optimization dictionary

The optimization dictionary contains all the settings compulsory for the optimization algorithm in order to run a proper optimization. 

In [2]:
optimization_client_name = "test_dCRAB_2_control_fields"

In [3]:
optimization_dictionary = {"optimization_client_name": optimization_client_name,
                           'opti_algorithm_name': 'dCRAB', 
                           'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABAlgorithm', 
                           'opti_algorithm_class': 'DCrabAlgorithm', 
                          }

Number of iteration and super-iterations

In [4]:
# Total number of dCRAB superiteration
super_iteration_number = 5
# Maximum number of iteration per super-iteration
maximum_function_evaluations_number = 200

In [5]:
optimization_dictionary['algorithm_settings'] = { 'super_iteration_number': super_iteration_number, 
                                                 'maximum_function_evaluations_number': maximum_function_evaluations_number
                                                  }

Settings for the inner algorithm used by dCRAB

In [6]:
optimization_dictionary["dsm_settings"] = {'general_settings': {'dsm_name': 'nelder_mead', 'is_adaptive': False}, 
                                    'stopping_criteria': {'iterations_number': 100, 'xatol': 1e-10, 'frtol': 1e-10}}

### Times

We can define here how many times we need to use in the optimization

In [7]:
time_p = {'time_name': 'time_p', 'initial_value': 201e-9}

In [8]:
optimization_dictionary["times"] = [time_p]

### Parameters

In [9]:
optimization_dictionary['parameters'] = []

### Pulses

In [10]:
pulse_amplitude = {'pulse_name': 'Amplitude', 
                   'upper_limit': 0.25, 
                   'lower_limit': -0.25, 
                   'bins_number': 1001, 
                   'time_name': 'time_p', 
                   'amplitude_variation': 0.04, 
                   'basis': {'basis_name': 'Fourier', 
                             'basis_class': 'Fourier', 
                             'basis_module': 'quocslib.pulses.basis.Fourier', 
                             'basis_vector_number': 2, 
                             'random_super_parameter_distribution': 
                             {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                   'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                   'lower_limit': 0.01, 'upper_limit': 7.0}
                            }, 
                   'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                   'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: np.pi/3 + 0.0*t'}
                  }

In [11]:
pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': 10, 
               'lower_limit': -10, 
               'bins_number': 1001, 
               'time_name': 'time_p', 
               'amplitude_variation': 0.5, 
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': 2, 
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                          'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                          'lower_limit': 0.01, 'upper_limit': 7.0}}, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0*t'}
              }

In [12]:
optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]

### Put all together and get ready to start the optimization with Qudi-QuOCS

In [13]:
opti_comm_dict = {"optimization_dictionary": optimization_dictionary}

Load the optimization algorithm into the optimization logic and display it into the GUI

In [14]:
optimizationlogic.load_opti_comm_dict(opti_comm_dict)

#### Important: If the GUI is not showing the optimization dictionary, restart the Kernel

Print the optimization dictionary also here

In [ ]:
optimalcontrol.opti_comm_dict

In [16]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# runtime of the experiment
runtime = 5

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True

# Dictonary containing the general measurement parameters for the predefined measurements
globaldict = pulsedmasterlogic.generation_parameters
globaldict["laser_channel"] = "d_ch1" 
globaldict["sync_channel"] = "d_ch2" 
globaldict["gate_channel"] = "d_ch1" 
globaldict["microwave_channel"] = "a_ch1"
globaldict["microwave_amplitude"] = 0
globaldict["microwave_frequency"] = 1.586462e9
globaldict["rabi_period"] = 71.55e-9
globaldict["laser_length"] = 12e-06
globaldict["laser_delay"] = 0
globaldict["wait_time"] = 6e-06
globaldict["analog_trigger_voltage"] = 0.0

# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(globaldict)

# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)

# make sure everything is finished
time.sleep(5)

# how the laser response is analyzed    
pulsedmasterlogic.set_analysis_settings(method = 'mean_ref_nvision')
pulsedmasterlogic.set_analysis_settings(signal_start = 656e-9)
pulsedmasterlogic.set_analysis_settings(signal_end = 13.07e-6)
pulsedmasterlogic.set_analysis_settings(norm_start = 13.152e-6)
pulsedmasterlogic.set_analysis_settings(norm_end = 13.92e-6)
pulsedmasterlogic.set_extraction_settings(method = 'hand_through')


# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
qubit = OneQubit2Fields()

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30:
        print("Problem at the beginning! Surpassed the 30 secs")
        break

# print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
while  optimizationlogic.handle_exit_obj.is_user_running == True:
    # wait until QuOCS optimizes the controls
    # print("Wait until the controls logic gives the controls")
    while not controlslogic.are_pulses_calculated:
        time.sleep(0.1)
        # If the waiting time exceed 10 seconds left stop the optimization
        if time.time() - last_time_fom > 20:
            print("Too much time... Exit!")
            optimizationlogic.handle_exit_obj.is_user_running = False
            break
            
    # If time exceeded, exit from the while loop and give a very high fom to the optimization algorithm
    # and status code -1, to interrupt the optimization smoothly
    if optimizationlogic.handle_exit_obj.is_user_running == False:
        print('Stopping the measurement!')
        # Update the pulse with the maximum number
        fomlogic.update_fom(10**10, status_code=-1)
        break
    #######################################################################################################
    # Get the Controls
    #######################################################################################################
    # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
    controlslogic.are_pulses_calculated = False
    # Get the controls from the controls logic
    pulses, timegrids, parameters = controlslogic.pulses, controlslogic.timegrids, controlslogic.parameters
    #######################################################################################################
    # Perform the measurement
    #######################################################################################################
    # save the pulses as .txt files (predefined methods doesn't allow us to upload a numpy array as 
    # parameter)
    np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\amplitude.txt',np.column_stack((timegrids[0], pulses[0])))
    np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\phase.txt',np.column_stack((timegrids[0], pulses[1])))
    time.sleep(0.1)
    
    # Dictonary containing the measurement parameters of the specific sequence
    dictparamsall = pulsedmasterlogic.generate_method_params
    dictparams_oc = dictparamsall["quocs_pi_optimization"]
    dictparams_oc["name"] = 'quocs' # load it into the second segment of the AWG
    dictparams_oc["length"] = 200e-9
    dictparams_oc["filename_amplitude"] = r"amplitude"
    dictparams_oc["filename_phase"] = r"phase"
    dictparams_oc["folder_path"] = r"C:\Users\Mesoscopic\Desktop\quocs_pulses"
    dictparams_oc["num_of_points"] = 1 

    # generate the sequence
    pulsedmasterlogic.generate_predefined_sequence('quocs_pi_optimization',kwarg_dict = dictparams_oc)

    # sample & upload the sequence
    time.sleep(1)

    pulsedmasterlogic.sample_ensemble(dictparams_oc["name"],True)
    while pulsedmasterlogic.status_dict['sampload_busy']:
        
        time.sleep(0.5)
        
        #print('Uploading sequence ', dictparams_oc["name"],  '...')
        
        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False:
            break   

    #print('Finished uploading', dictparams_oc["name"], '!')
         
    # make sure everything is finished (crucial)
    time.sleep(2)#time.sleep(5)    

#########################################################################
#should work but dont test remotely
####################################################################
    # start the measurement
    print('Starting the measurement!')
    pulsedmasterlogic.toggle_pulsed_measurement(True)
    time.sleep(1)
    
    # make sure the measurement started
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)

    measurement_start_time = time.time()
    
    while time.time() <= measurement_start_time + runtime:
        time.sleep(0.2)
        
        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False:
            print('Stopping the measurement!')
            break
       
    # option to stop the measurement
    if pulsedmasterlogic.globalrun == False:
        print('Stopping the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(False)
        break
        
    time.sleep(1)
            
    # Stop the measurement
    pulsedmasterlogic.toggle_pulsed_measurement(False)
    
    # Make sure it stopped (Wait until the Picoscope card sends the data)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)

    # calculate the fom
    fom = pulsedmeasurementlogic.signal_data[1,:][1] - pulsedmeasurementlogic.signal_data[1,:][0]
        
    #print('Measurement finished!')       

    #
    #######################################################################################################
    # Calculate the figure of merit
    #######################################################################################################
    # calculate the Figure of Merit
    #fom = qubit.get_FoM(pulses, timegrids, parameters)["FoM"]
    #######################################################################################################
    # Return the figure of merit
    #######################################################################################################
    # Update the figure of merit to the fom logic
    fomlogic.update_fom(fom, status_code=0)
    # update the last time the fom is calculated
    last_time_fom = time.time()
    #######################################################################################################
    # Optional part
    #######################################################################################################
    # Print the data just for debug purpose
    print("FoM: {0}, status_code: {1}".format(fom, 0))
print("Optimization finished")

Starting the measurement!
FoM: -0.015289641602592607, status_code: 0
Starting the measurement!
FoM: -0.016437891581778552, status_code: 0
Starting the measurement!
FoM: -0.018781177891339595, status_code: 0
Starting the measurement!
FoM: -0.017883232213198985, status_code: 0
Starting the measurement!
FoM: -0.017567923961351206, status_code: 0
Starting the measurement!
FoM: -0.016458923616951115, status_code: 0
Starting the measurement!
FoM: -0.01442441935783667, status_code: 0
Starting the measurement!
FoM: -0.006167154415236742, status_code: 0
Starting the measurement!
FoM: -0.02023123944428573, status_code: 0
Starting the measurement!
FoM: -0.016903885371748695, status_code: 0
Starting the measurement!
FoM: -0.022709683472993736, status_code: 0
Starting the measurement!
FoM: -0.009559524768762984, status_code: 0
Starting the measurement!
FoM: -0.02069061831181762, status_code: 0
Starting the measurement!
FoM: -0.021079647401554102, status_code: 0
Starting the measurement!
FoM: -0.021

Starting the measurement!
FoM: -0.02253378431093389, status_code: 0
Starting the measurement!
FoM: -0.022399759366909522, status_code: 0
Starting the measurement!
FoM: -0.022444314038868374, status_code: 0
Starting the measurement!
FoM: -0.022548798641316647, status_code: 0
Starting the measurement!
FoM: -0.022469423670288813, status_code: 0
Starting the measurement!
FoM: -0.022455547803809095, status_code: 0
Starting the measurement!
FoM: -0.022710879993782962, status_code: 0
Starting the measurement!
FoM: -0.0225023363249518, status_code: 0
Starting the measurement!
FoM: -0.02248971805259492, status_code: 0
Starting the measurement!
FoM: -0.022667415138098446, status_code: 0
Starting the measurement!
FoM: -0.022441470902963445, status_code: 0
Starting the measurement!
FoM: -0.022570340886173823, status_code: 0
Starting the measurement!
FoM: -0.02244429597580011, status_code: 0
Starting the measurement!
FoM: -0.022512403191679464, status_code: 0
Starting the measurement!
FoM: -0.02277

Starting the measurement!
FoM: -0.02204437592804298, status_code: 0
Starting the measurement!
FoM: -0.022203069797011366, status_code: 0
Starting the measurement!
FoM: -0.020561749918715067, status_code: 0
Starting the measurement!
FoM: -0.02178597905837598, status_code: 0
Starting the measurement!
FoM: -0.021642018204809155, status_code: 0
Starting the measurement!
FoM: -0.0221762976665405, status_code: 0
Starting the measurement!
FoM: -0.02191877172586354, status_code: 0
Starting the measurement!
FoM: -0.02211432425953641, status_code: 0
Starting the measurement!
FoM: -0.02197061600493211, status_code: 0
Starting the measurement!
FoM: -0.02177360320804178, status_code: 0
Starting the measurement!
FoM: -0.02165930111287473, status_code: 0
Starting the measurement!
FoM: -0.021879118599432368, status_code: 0
Starting the measurement!
FoM: -0.021924975873938712, status_code: 0
Starting the measurement!
FoM: -0.021797533003713454, status_code: 0
Starting the measurement!
FoM: -0.021954152

Starting the measurement!
FoM: -0.021690392940347802, status_code: 0
Starting the measurement!
FoM: -0.021762098128263574, status_code: 0
Starting the measurement!
FoM: -0.021750865710627787, status_code: 0
Starting the measurement!
FoM: -0.02166640224899563, status_code: 0
Starting the measurement!
FoM: -0.02227739454247868, status_code: 0
Starting the measurement!
FoM: -0.021668711656369077, status_code: 0
Starting the measurement!
FoM: -0.022022201462430058, status_code: 0
Starting the measurement!
FoM: -0.021623644299494038, status_code: 0
Starting the measurement!
FoM: -0.02183746767444361, status_code: 0
Starting the measurement!
FoM: -0.02159049541326441, status_code: 0
Starting the measurement!
FoM: -0.021644327325214374, status_code: 0
Starting the measurement!
FoM: -0.021760194280453726, status_code: 0
Starting the measurement!
FoM: -0.021694115509375722, status_code: 0
Starting the measurement!
FoM: -0.021671693576790774, status_code: 0
Starting the measurement!
FoM: -0.0217

FoM: -0.021423635037588373, status_code: 0
Starting the measurement!
FoM: -0.021461246006159285, status_code: 0
Starting the measurement!
FoM: -0.021542329440551122, status_code: 0
Starting the measurement!
FoM: -0.021418339332528902, status_code: 0
Starting the measurement!
FoM: -0.02146116900169992, status_code: 0
Starting the measurement!
FoM: -0.021524557990992066, status_code: 0
Starting the measurement!
FoM: -0.021521507619704172, status_code: 0
Starting the measurement!
FoM: -0.02133543559014539, status_code: 0
Starting the measurement!
FoM: -0.021086654365863433, status_code: 0
Starting the measurement!
FoM: -0.02127168524424039, status_code: 0
Starting the measurement!
FoM: -0.021771613102073095, status_code: 0
Starting the measurement!
FoM: -0.021372800681019433, status_code: 0
Starting the measurement!
FoM: -0.021395710773635557, status_code: 0
Starting the measurement!
FoM: -0.021175869453243457, status_code: 0
Starting the measurement!
FoM: -0.021630891795674123, status_co

Starting the measurement!
FoM: -0.019747066437212357, status_code: 0
Starting the measurement!
FoM: -0.020153159959657718, status_code: 0
Starting the measurement!
FoM: -0.02000390602548585, status_code: 0
Starting the measurement!
FoM: -0.020075449021464187, status_code: 0
Starting the measurement!
FoM: -0.01976368347208779, status_code: 0
Starting the measurement!
FoM: -0.02004942970132073, status_code: 0
Starting the measurement!
FoM: -0.01999029352579662, status_code: 0
Starting the measurement!
FoM: -0.019701556783618956, status_code: 0
Starting the measurement!
FoM: -0.020047784967807014, status_code: 0
Starting the measurement!
FoM: -0.020046757348541666, status_code: 0
Starting the measurement!
FoM: -0.019385199764999417, status_code: 0
Starting the measurement!
FoM: -0.01509765438975963, status_code: 0
Starting the measurement!
FoM: -0.019270268121475587, status_code: 0
Starting the measurement!
FoM: -0.018197085673630098, status_code: 0
Starting the measurement!
FoM: -0.01709

In [ ]:
# Access to the optimizer object to get info about the optimization
optimizer_obj = optimizationlogic.optimizer_obj

In [ ]:
optimizer_obj

In [ ]:
optimizer_obj.best_fom

In [ ]:
# Best controls
pulses_list, time_grids_list, parameters_list = optimizer_obj.get_best_controls()
t_amplitude = time_grids_list[0]
amplitude = pulses_list[0]
t_phase = time_grids_list[1]
phase = pulses_list[1]